In [1]:
# Electricity & Gas Consumption Heat Map by California Counties
# Energy Usage Data Resource from California Energy Commission (https://www.energy.ca.gov/)
# County Border Resource from Opendatasoft (https://hifld-dhs-gii.opendata.arcgis.com/datasets/53270fe7036f428fbfb11c4511994a6f_0)

# Importing Block
import geopandas as gpd
import pandas as pd
import numpy as np
import json
import folium

In [82]:
# All Usage Expressed in Millions of kWh (GWh)
# All electricity usages are total of residential and non-residential

###### Note
"""
1. Keep thinking about what needs to be done
2. Make county border lines and a county marker including a tooltip that contains an Total of and an average of energy usages + energy heatmap indicator

- Add Geo Points for each county
- Extract values from 'coordinates' (json? dict?, type = np.ndarray with one dimension) and add to the map

"""
######


### Data Path Block
usage_df = pd.read_csv('/Users/jk/Desktop/A/Study/github/Data-Analysis-Project/Energy Consumption Heatmap by California Counties/ElectricityByCounty_1990_2019.csv')
county_df = pd.read_csv('/Users/jk/Desktop/A/Study/github/Data-Analysis-Project/Energy Consumption Heatmap by California Counties/us-county-boundaries.csv', sep = ';')


### Data Formatting Block
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

usage_df.drop(columns = ['Sector'], 
              inplace = True) # To drop 'Sector' (value: Total) column

years = list(map(str, range(1990, 2020))) # Convert columns of each year to str value for line ???
usage_df[years] = usage_df[years].astype(int)
usage_df['Total Usage'] = usage_df['Total Usage'].astype(int) 

ca_county = county_df[county_df['STUSAB'] == 'CA']
ca_county = ca_county[['NAME', 
                       'Geo Point', 
                       'Geo Shape']].sort_values(by = 'NAME')
ca_county.set_index([pd.Index(list(map(int, range(1, 59))))], inplace = True) # Add index 1 - 58



### Folium Setting Block
ca_map = folium.Map(location = [36.7783, -119.4179],
                    width = 640, 
                    height = 700, 
                    zoom_start = 6, 
                    min_zoom = 6) # California lat, long


for x in ca_county['Geo Point'][1].split(','):
    print(x)

# for name in ca_county['NAME']:
    # county_marker = foliumn.Marker(location = [ , ], popup = f'<i>{name}<\i>').add_to(ca_map)


# ccc_marker = folium.Marker(location = [37.8534, -121.9018], popup = '<i>Contra Costa County</i>', tooltip = 'Contra Costa County').add_to(ca_map)
# # contra_costa_border = folium.Popup(locations = [aa]).add_to(ca_map)

display(type(ca_county['Geo Point'][1].split()))



37.6504930265
-121.917998966


list